In [1]:
import keras
import numpy as np
from keras.layers import Dense, Activation, Dropout
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.optimizers import RMSprop

/Users/viki/anaconda3/lib/python3.5/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
/Users/viki/anaconda3/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: compiletime version 3.6 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.5
  return f(*args, **kwds)


In [2]:
# lets limit input to 16 digits
INPUT_DIGITS = 16
number_of_numbers = 2 ** INPUT_DIGITS - 1
input_number = []
output_label = []
for i in range(1, number_of_numbers):
    input_number.append(i)
    if i % 3 == 0 and i % 5 == 0:
        output_label.append(0) # fizzbuzz
    elif i % 3 == 0:
        output_label.append(1) # fizz
    elif i % 5 == 0:
        output_label.append(2) # buzz
    else:
        output_label.append(3) # Same

In [3]:
y = keras.utils.to_categorical(output_label)

In [4]:
def map_input_numer_to_binary(num):
    binary_rep = list("{0:b}".format(num))
    final_res = ['0' for _ in range(INPUT_DIGITS)]
    for idx, v in enumerate(reversed(binary_rep)):
        final_res[idx] = v
    return list(reversed(final_res))

In [5]:
def reverse_map_binary_to_number(binary):
    return int(''.join(binary), 2)

In [6]:
print(map_input_numer_to_binary(2), reverse_map_binary_to_number(map_input_numer_to_binary(2)))

['0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '1', '0'] 2


In [7]:
x = np.array([map_input_numer_to_binary(v) for v in input_number])

In [8]:
x_train, x_val, y_train, y_val = train_test_split(x, y, random_state = 3)

In [9]:
x_train.shape, x_val.shape, y_train.shape, y_val.shape

((49150, 16), (16384, 16), (49150, 4), (16384, 4))

In [10]:
model = Sequential()

model.add(Dense(64, input_shape=(16,)))
model.add(keras.layers.advanced_activations.LeakyReLU(alpha=0.3))
# model.add(Dropout(0.2))
model.add(Dense(128))
model.add(keras.layers.advanced_activations.LeakyReLU(alpha=0.3))
# model.add(Dropout(0.2))
model.add(Dense(4))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(x_train, y_train, epochs=100, batch_size=128, validation_split=0.1)

Train on 44235 samples, validate on 4915 samples
Epoch 1/100
44235/44235 [==============================] - 2s 38us/step - loss: 1.1500 - acc: 0.5303 - val_loss: 1.1552 - val_acc: 0.5284
Epoch 2/100
44235/44235 [==============================] - 1s 25us/step - loss: 1.1406 - acc: 0.5337 - val_loss: 1.1521 - val_acc: 0.5284
Epoch 3/100
44235/44235 [==============================] - 1s 32us/step - loss: 1.1385 - acc: 0.5337 - val_loss: 1.1524 - val_acc: 0.5284
Epoch 4/100
44235/44235 [==============================] - 1s 31us/step - loss: 1.1381 - acc: 0.5337 - val_loss: 1.1515 - val_acc: 0.5284
Epoch 5/100
44235/44235 [==============================] - 1s 25us/step - loss: 1.1366 - acc: 0.5337 - val_loss: 1.1475 - val_acc: 0.5284
Epoch 6/100
44235/44235 [==============================] - 1s 25us/step - loss: 1.1339 - acc: 0.5337 - val_loss: 1.1475 - val_acc: 0.5284
Epoch 7/100
44235/44235 [==============================] - 1s 26us/step - loss: 1.1307 - acc: 0.5337 - val_loss: 1.1420 - v

44235/44235 [==============================] - 1s 25us/step - loss: 0.0456 - acc: 0.9889 - val_loss: 0.0511 - val_acc: 0.9858
Epoch 60/100
44235/44235 [==============================] - 1s 24us/step - loss: 0.0472 - acc: 0.9888 - val_loss: 0.0461 - val_acc: 0.9888
Epoch 61/100
44235/44235 [==============================] - 1s 24us/step - loss: 0.0425 - acc: 0.9908 - val_loss: 0.0352 - val_acc: 0.9921
Epoch 62/100
44235/44235 [==============================] - 1s 27us/step - loss: 0.0447 - acc: 0.9889 - val_loss: 0.0323 - val_acc: 0.9929
Epoch 63/100
44235/44235 [==============================] - 1s 26us/step - loss: 0.0411 - acc: 0.9903 - val_loss: 0.0493 - val_acc: 0.9860
Epoch 64/100
44235/44235 [==============================] - 1s 27us/step - loss: 0.0427 - acc: 0.9900 - val_loss: 0.0361 - val_acc: 0.9915
Epoch 65/100
44235/44235 [==============================] - 1s 27us/step - loss: 0.0400 - acc: 0.9909 - val_loss: 0.0331 - val_acc: 0.9935
Epoch 66/100
44235/44235 [==============

In [11]:
predictions = model.predict(x_val)

In [12]:
from sklearn.metrics import accuracy_score

In [13]:
accuracy_score(np.argmax(predictions, axis=1), np.argmax(y_val, axis=1))

0.9974365234375

In [14]:
def print_sample(input_value, output_prediction, expected_ouput, no_of_sampels=5, print_values=False):
    correct = 0
    count = 0
    for value, pred, expected in zip(input_value, output_prediction, expected_ouput):
        val = reverse_map_binary_to_number(value)
        pred_val = np.argmax(pred)
        expected_val = np.argmax(expected)
        if print_values: print(val, pred_val, expected_val) # To prevent printing all values
        if expected_val == pred_val:
            correct += 1
        count += 1
        if no_of_sampels and count >= no_of_sampels:
            break
    print('correct', correct/count)

In [15]:
print_sample(x_val, predictions, y_val, 10, True)

56698 3 3
27266 3 3
62738 3 3
58929 1 1
50762 3 3
59504 3 3
57471 1 1
27106 3 3
18344 3 3
61343 3 3
correct 1.0
